In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import duckdb
# from pathlib import Path

DATA_PATH = "../data/processed/citibike/*/*/data.parquet"
# OUT_PATH = "../data/processed/station_hour.parquet"

#Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)

query = f"""
WITH trips AS (
    SELECT
        start_station_name AS station,
        date_trunc('hour', started_at) AS hour
    FROM read_parquet('{DATA_PATH}')
    WHERE started_at IS NOT NULL
),
bounds AS (
    SELECT
        min(hour) AS min_hour,
        max(hour) AS max_hour
    FROM trips
),
hours AS (
    -- build a row for every hour between min_hour and max_hour
    SELECT
        min_hour + (i || ' hours')::INTERVAL AS hour
    FROM bounds,
         range(date_diff('hour', min_hour, max_hour) + 1) AS t(i)
),
stations AS (
    SELECT DISTINCT station
    FROM trips
),
grid AS (
    -- dense grid: every station × every hour
    SELECT
        s.station,
        h.hour
    FROM stations s
    CROSS JOIN hours h
),
counts AS (
    -- actual observed counts per station-hour
    SELECT
        station,
        hour,
        COUNT(*) AS n_trips
    FROM trips
    GROUP BY station, hour
)
SELECT
    g.station AS start_station_name,
    g.hour,
    COALESCE(c.n_trips, 0) AS n_trips
FROM grid g
LEFT JOIN counts c
USING (station, hour)
ORDER BY start_station_name, hour
"""

df_station_hour = duckdb.query(query).df()
# df_station_hour.to_parquet(OUT_PATH, index=False)

df_station_hour.head()


,start_station_name,hour,n_trips
0,1 Ave & E 110 St,2022-12-28 09:00:00,0
1,1 Ave & E 110 St,2022-12-28 10:00:00,0
2,1 Ave & E 110 St,2022-12-28 11:00:00,0
3,1 Ave & E 110 St,2022-12-28 12:00:00,0
4,1 Ave & E 110 St,2022-12-28 13:00:00,0


In [7]:
df_station_hour.describe()

,hour,n_trips
count,61195785,6.119578e+07
mean,2024-05-30 16:00:00.000002,1.949477e+00
min,2022-12-28 09:00:00,0.000000e+00
25%,2023-09-14 00:00:00,0.000000e+00
50%,2024-05-30 16:00:00,0.000000e+00
75%,2025-02-14 08:00:00,2.000000e+00
max,2025-10-31 23:00:00,2.140000e+02
std,NaN,4.393544e+00


In [8]:
df_station_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61195785 entries, 0 to 61195784
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_station_name  object        
 1   hour                datetime64[us]
 2   n_trips             int64         
dtypes: datetime64[us](1), int64(1), object(1)
memory usage: 1.4+ GB


In [11]:
import duckdb
from pathlib import Path

DATA_PATH = "../data/processed/citibike/*/*/data.parquet"
# OUT_PATH = "../data/processed/station_hour.parquet"

# Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)

query = f"""
WITH trips AS (
    SELECT
        start_station_name AS station,
        date_trunc('hour', started_at) AS hour
    FROM read_parquet('{DATA_PATH}')
    WHERE started_at IS NOT NULL
    AND started_at >= '2025-01-01'
),
bounds AS (
    SELECT
        min(hour) AS min_hour,
        max(hour) AS max_hour
    FROM trips
),
hours AS (
    -- build a row for every hour between min_hour and max_hour
    SELECT
        min_hour + (i || ' hours')::INTERVAL AS hour
    FROM bounds,
         range(date_diff('hour', min_hour, max_hour) + 1) AS t(i)
),
stations AS (
    SELECT DISTINCT station
    FROM trips
),
grid AS (
    -- dense grid: every station × every hour
    SELECT
        s.station,
        h.hour
    FROM stations s
    CROSS JOIN hours h
),
counts AS (
    -- actual observed counts per station-hour
    SELECT
        station,
        hour,
        COUNT(*) AS n_trips
    FROM trips
    GROUP BY station, hour
)
SELECT
    g.station AS start_station_name,
    g.hour,
    COALESCE(c.n_trips, 0) AS n_trips,

    -- time features
    CAST(strftime(g.hour, '%Y') AS INTEGER) AS year,
    CAST(strftime(g.hour, '%m') AS INTEGER) AS month,
    CAST(strftime(g.hour, '%d') AS INTEGER) AS day_of_month,
    CAST(strftime(g.hour, '%w') AS INTEGER) AS day_of_week,       -- 0=Sunday,...,6=Saturday
    CAST(strftime(g.hour, '%H') AS INTEGER) AS hour_of_day,       -- 0-23
    CASE 
        WHEN CAST(strftime(g.hour, '%w') AS INTEGER) IN (0, 6) 
        THEN 1 ELSE 0 
    END AS is_weekend
FROM grid g
LEFT JOIN counts c
USING (station, hour)
ORDER BY start_station_name, hour
"""

df_station_hour = duckdb.query(query).df()
#df_station_hour.to_parquet(OUT_PATH, index=False)

df_station_hour.head()


,start_station_name,hour,n_trips,year,month,day_of_month,day_of_week,hour_of_day,is_weekend
0,1 Ave & E 110 St,2025-01-01 00:00:00,0,2025,1,1,3,0,0
1,1 Ave & E 110 St,2025-01-01 01:00:00,3,2025,1,1,3,1,0
2,1 Ave & E 110 St,2025-01-01 02:00:00,0,2025,1,1,3,2,0
3,1 Ave & E 110 St,2025-01-01 03:00:00,0,2025,1,1,3,3,0
4,1 Ave & E 110 St,2025-01-01 04:00:00,1,2025,1,1,3,4,0


In [9]:
import duckdb
# from pathlib import Path

DATA_PATH = "../data/processed/citibike/*/*/data.parquet"
# OUT_PATH = "../data/processed/station_hour.parquet"

# Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)

query = f"""
WITH trips AS (
    SELECT
        start_station_name AS station,
        date_trunc('hour', started_at) AS hour
    FROM read_parquet('{DATA_PATH}')
    WHERE started_at IS NOT NULL
    AND started_at >= '2023-01-01'
),
bounds AS (
    SELECT
        min(hour) AS min_hour,
        max(hour) AS max_hour
    FROM trips
),
hours AS (
    -- build a row for every hour between min_hour and max_hour
    SELECT
        min_hour + (i || ' hours')::INTERVAL AS hour
    FROM bounds,
         range(date_diff('hour', min_hour, max_hour) + 1) AS t(i)
),
stations AS (
    SELECT DISTINCT station
    FROM trips
),
grid AS (
    -- dense grid: every station × every hour
    SELECT
        s.station,
        h.hour
    FROM stations s
    CROSS JOIN hours h
),
counts AS (
    -- actual observed counts per station-hour
    SELECT
        station,
        hour,
        COUNT(*) AS n_trips
    FROM trips
    GROUP BY station, hour
),
base AS (
    -- base station-hour table with time features
    SELECT
        g.station AS start_station_name,
        g.hour,
        COALESCE(c.n_trips, 0) AS n_trips,

        -- time features
        CAST(strftime(g.hour, '%Y') AS INTEGER) AS year,
        CAST(strftime(g.hour, '%m') AS INTEGER) AS month,
        CAST(strftime(g.hour, '%d') AS INTEGER) AS day_of_month,
        CAST(strftime(g.hour, '%w') AS INTEGER) AS day_of_week,    -- 0=Sunday,...,6=Saturday
        CAST(strftime(g.hour, '%H') AS INTEGER) AS hour_of_day,    -- 0–23
        CASE
            WHEN CAST(strftime(g.hour, '%w') AS INTEGER) IN (0, 6)
            THEN 1 ELSE 0
        END AS is_weekend
    FROM grid g
    LEFT JOIN counts c
    USING (station, hour)
),
final AS (
    SELECT
        *,
        -- 1-hour lag
        LAG(n_trips, 1) OVER (
            PARTITION BY start_station_name
            ORDER BY hour
        ) AS n_trips_lag_1h,

        -- 24-hour lag (same station, previous day same hour)
        LAG(n_trips, 24) OVER (
            PARTITION BY start_station_name
            ORDER BY hour
        ) AS n_trips_lag_24h,

        -- mean over last 6 hours (excluding current hour)
        AVG(n_trips) OVER (
            PARTITION BY start_station_name
            ORDER BY hour
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS n_trips_mean_6h,

        -- mean over last 24 hours (excluding current hour)
        AVG(n_trips) OVER (
            PARTITION BY start_station_name
            ORDER BY hour
            ROWS BETWEEN 24 PRECEDING AND 1 PRECEDING
        ) AS n_trips_mean_24h
    FROM base
)
SELECT
    *
FROM final
ORDER BY start_station_name, hour
"""

df_station_hour = duckdb.query(query).df()
# df_station_hour.to_parquet(OUT_PATH, index=False)

df_station_hour.head()


,start_station_name,hour,n_trips,year,month,day_of_month,day_of_week,hour_of_day,is_weekend,n_trips_lag_1h,n_trips_lag_24h,n_trips_mean_6h,n_trips_mean_24h
0,1 Ave & E 110 St,2023-01-01 00:00:00,0,2023,1,1,0,0,1,<NA>,<NA>,NaN,NaN
1,1 Ave & E 110 St,2023-01-01 01:00:00,1,2023,1,1,0,1,1,0,<NA>,0.000000,0.000000
2,1 Ave & E 110 St,2023-01-01 02:00:00,0,2023,1,1,0,2,1,1,<NA>,0.500000,0.500000
3,1 Ave & E 110 St,2023-01-01 03:00:00,1,2023,1,1,0,3,1,0,<NA>,0.333333,0.333333
4,1 Ave & E 110 St,2023-01-01 04:00:00,0,2023,1,1,0,4,1,1,<NA>,0.500000,0.500000


In [10]:
df_station_hour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58175280 entries, 0 to 58175279
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_station_name  object        
 1   hour                datetime64[us]
 2   n_trips             int64         
 3   year                int32         
 4   month               int32         
 5   day_of_month        int32         
 6   day_of_week         int32         
 7   hour_of_day         int32         
 8   is_weekend          int32         
 9   n_trips_lag_1h      Int64         
 10  n_trips_lag_24h     Int64         
 11  n_trips_mean_6h     float64       
 12  n_trips_mean_24h    float64       
dtypes: Int64(2), datetime64[us](1), float64(2), int32(6), int64(1), object(1)
memory usage: 4.4+ GB


In [11]:
df_model = df_station_hour.dropna(
    subset=[
        "n_trips_lag_1h",
        "n_trips_lag_24h",
        "n_trips_mean_6h",
        "n_trips_mean_24h",
    ]
).copy()


In [12]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58119072 entries, 24 to 58175279
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   start_station_name  object        
 1   hour                datetime64[us]
 2   n_trips             int64         
 3   year                int32         
 4   month               int32         
 5   day_of_month        int32         
 6   day_of_week         int32         
 7   hour_of_day         int32         
 8   is_weekend          int32         
 9   n_trips_lag_1h      Int64         
 10  n_trips_lag_24h     Int64         
 11  n_trips_mean_6h     float64       
 12  n_trips_mean_24h    float64       
dtypes: Int64(2), datetime64[us](1), float64(2), int32(6), int64(1), object(1)
memory usage: 4.9+ GB


In [13]:
df_model.isnull().sum()

start_station_name    0
hour                  0
n_trips               0
year                  0
month                 0
day_of_month          0
day_of_week           0
hour_of_day           0
is_weekend            0
n_trips_lag_1h        0
n_trips_lag_24h       0
n_trips_mean_6h       0
n_trips_mean_24h      0
dtype: int64

In [14]:
df = df_model

df["station_id"] = df["start_station_name"].astype("category").cat.codes


train = df[df["hour"] < "2025-07-01"]
val   = df[(df["hour"] >= "2025-07-01") & (df["hour"] < "2025-10-01")]
test  = df[df["hour"] >= "2025-10-01"]

len(train), len(val), len(test)


(51205488, 5171136, 1742448)

In [15]:
import catboost as cb
features = [
    "station_id",
    "year",
    "month",
    "day_of_month",
    "day_of_week",
    "hour_of_day",
    "is_weekend",
    "n_trips_lag_1h",
    "n_trips_lag_24h",
    "n_trips_mean_6h",
    "n_trips_mean_24h",
]
target = "n_trips"
train_pool = cb.Pool(train[features], train[target], cat_features=["station_id"])
val_pool   = cb.Pool(val[features], val[target], cat_features=["station_id"])
test_pool  = cb.Pool(test[features], test[target], cat_features=["station_id"])
model = cb.CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    eval_metric="RMSE",
    random_seed=42,
    early_stopping_rounds=50,
    verbose=100,
)
model.fit(train_pool, eval_set=val_pool)
preds = model.predict(test_pool)
from sklearn.metrics import mean_squared_error
import numpy as np
rmse = np.sqrt(mean_squared_error(test[target], preds))
print(f"Test RMSE: {rmse:.4f}")


0:	learn: 3.9841343	test: 5.5547058	best: 5.5547058 (0)	total: 3.69s	remaining: 1h 1m 21s
100:	learn: 2.0223869	test: 2.6651583	best: 2.6651583 (100)	total: 5m 39s	remaining: 50m 20s
200:	learn: 1.9761411	test: 2.6021263	best: 2.6021263 (200)	total: 10m 56s	remaining: 43m 30s
300:	learn: 1.9562969	test: 2.5763080	best: 2.5763080 (300)	total: 16m 5s	remaining: 37m 23s
400:	learn: 1.9434841	test: 2.5596972	best: 2.5596972 (400)	total: 20m 49s	remaining: 31m 7s
500:	learn: 1.9347452	test: 2.5490807	best: 2.5490591 (497)	total: 25m 55s	remaining: 25m 48s
600:	learn: 1.9272863	test: 2.5407247	best: 2.5407041 (597)	total: 30m 40s	remaining: 20m 21s
700:	learn: 1.9211731	test: 2.5333850	best: 2.5333850 (700)	total: 35m 15s	remaining: 15m 2s
800:	learn: 1.9160613	test: 2.5269345	best: 2.5269345 (800)	total: 39m 43s	remaining: 9m 52s
900:	learn: 1.9114461	test: 2.5223315	best: 2.5223315 (900)	total: 44m 24s	remaining: 4m 52s
999:	learn: 1.9072229	test: 2.5185326	best: 2.5184401 (995)	total: 48m

In [19]:
from sklearn.metrics import r2_score
r2 = r2_score(test[target], preds)
print(f"Test R²: {r2:.4f}")

Test R²: 0.8185


In [22]:

station_ohe = pd.get_dummies(
    df["station_id"],
    prefix="station",
    sparse=True   # <- crucial for 2455 stations
)

X = pd.concat([df[features], station_ohe], axis=1)


In [ ]:
y_pred = test["n_trips_lag_24h"]
y_true = test["n_trips"]

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse:.2f} trips")
mae = (y_pred - y_true).abs().mean()
print(f"MAE: {mae:.2f} trips")
r2 = r2_score(y_true, y_pred)
print(f"R²: {r2:.4f}")

RMSE: 3.56 trips
MAE: 1.69 trips
R²: 0.5878


: 

In [18]:
test["n_trips"].median()

1.0

In [7]:
import xgboost as xgb

features = [
    "hour_of_day", "day_of_week", "is_weekend",
    "n_trips_lag_1h", "n_trips_lag_24h",
    "n_trips_mean_6h", "n_trips_mean_24h",
    "station_id",
]

dtrain = xgb.DMatrix(train[features], label=train["n_trips"])
dval   = xgb.DMatrix(val[features],   label=val["n_trips"])
dtest  = xgb.DMatrix(test[features],  label=test["n_trips"])

params = {
    "objective": "count:poisson",
    "eval_metric": "poisson-nloglik",
    "max_depth": 8,
    "eta": 0.1,
    "tree_method": "hist",
    "enable_categorical": True,
}

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtrain, "train"), (dval, "val")],
    early_stopping_rounds=50
)

pred_test = bst.predict(dtest)


/Users/jesper/Documents/AXA/DSC/.conda/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [10:18:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "enable_categorical" } are not used.

  self.starting_round = model.num_boosted_rounds()


KeyboardInterrupt: 

In [26]:
mae = (pred_test - test["n_trips"]).abs().mean()
print(f"MAE: {mae:.2f} trips")

MAE: 1.20 trips


In [23]:
print(df_station_hour['n_trips'].mean())
print(df_station_hour['n_trips'].median())

1.949476945185032
0.0


In [ ]:
wmape = test.abs_error.sum() / test.n_trips.sum()


In [9]:
import xgboost as xgb
print(xgb.__version__)
# print(xgb.Booster.params)


3.1.2


In [10]:
import xgboost as xgb
import numpy as np
import pandas as pd

df = pd.DataFrame({
    "cat": pd.Series(["a","b","c","a"], dtype="category"),
    "y": [1, 2, 3, 1]
})

dtrain = xgb.DMatrix(df[["cat"]], label=df["y"], enable_categorical=True)
